<a href="https://colab.research.google.com/github/DanielWarfield1/recalldemo1/blob/main/SampleCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making a Recall bot join, record, and getting the transcript

based on https://docs.recall.ai/docs/meeting-caption-transcription

first, join an instant meeting: https://meet.google.com/landing

In [ ]:
from google.colab import userdata
import requests
import json

RECALL_API_KEY = userdata.get('recall_api_key')

In [ ]:
meeting_url = "https://meet.google.com/ams-vsdo-ewq"

## Creating a Bot

In [ ]:
def start_recall_bot(api_key, meeting_url, region='us-west-2'):

    url = f"https://{region}.recall.ai/api/v1/bot/"

    payload = {
        "meeting_url": meeting_url,
        "bot_name": "demoBot",
        "recording_config": {"transcript": { "provider": {"meeting_captions":{}}}}
    }

    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": api_key
    }

    return requests.post(url, json=payload, headers=headers)

response = start_recall_bot(RECALL_API_KEY, meeting_url)
bot_id = json.loads(response.text)['id']
print(bot_id)

## Getting Transcript of conversation

In [ ]:
import requests

def get_transcript(api_key, bot_id, region='us-west-2'):
    url = f"https://{region}.recall.ai/api/v1/bot/{bot_id}/transcript/"

    headers = {
        "accept": "application/json",
        "Authorization": api_key
    }

    return requests.get(url, headers=headers)

response = get_transcript(RECALL_API_KEY, bot_id)
transcript = json.loads(response.text)
print(transcript)

[{'words': [{'text': "Hello, my name is Daniel Warfield and I'm talking to the Bot.", 'start_timestamp': 0.0, 'end_timestamp': 2.6631839275360107, 'language': None, 'confidence': None}], 'speaker': 'Daniel Warfield', 'speaker_id': 100, 'language': 'en-us'}, {'words': [{'text': 'Maybe.', 'start_timestamp': 9.075255393981934, 'end_timestamp': 9.728450775146484, 'language': None, 'confidence': None}], 'speaker': 'Daniel Warfield', 'speaker_id': 100, 'language': 'en-us'}, {'words': [{'text': 'Oh, thank God. Okay.', 'start_timestamp': 18.68805503845215, 'end_timestamp': 20.342729568481445, 'language': None, 'confidence': None}], 'speaker': 'Daniel Warfield', 'speaker_id': 100, 'language': 'en-us'}, {'words': [{'text': 'Cool. The money in the email.', 'start_timestamp': 29.976699829101562, 'end_timestamp': 32.7333869934082, 'language': None, 'confidence': None}], 'speaker': 'Daniel Warfield', 'speaker_id': 100, 'language': 'en-us'}, {'words': [{'text': 'Beautiful.', 'start_timestamp': 33.870

In [ ]:
transcript

[{'words': [{'text': "Hello, my name is Daniel Warfield and I'm talking to the Bot.",
    'start_timestamp': 0.0,
    'end_timestamp': 2.6631839275360107,
    'language': None,
    'confidence': None}],
  'speaker': 'Daniel Warfield',
  'speaker_id': 100,
  'language': 'en-us'},
 {'words': [{'text': 'Maybe.',
    'start_timestamp': 9.075255393981934,
    'end_timestamp': 9.728450775146484,
    'language': None,
    'confidence': None}],
  'speaker': 'Daniel Warfield',
  'speaker_id': 100,
  'language': 'en-us'},
 {'words': [{'text': 'Oh, thank God. Okay.',
    'start_timestamp': 18.68805503845215,
    'end_timestamp': 20.342729568481445,
    'language': None,
    'confidence': None}],
  'speaker': 'Daniel Warfield',
  'speaker_id': 100,
  'language': 'en-us'},
 {'words': [{'text': 'Cool. The money in the email.',
    'start_timestamp': 29.976699829101562,
    'end_timestamp': 32.7333869934082,
    'language': None,
    'confidence': None}],
  'speaker': 'Daniel Warfield',
  'speaker_id

# Unpacking into a Conversational Transcript

In [ ]:
current_speaker = None
current_language = None


for fragment in transcript:

    #getting a particular piece of the transcript
    fragment_speaker = fragment['speaker']
    fragment_language = fragment['language']

    #printing name and language, if changed
    if (fragment_speaker != current_speaker or
        fragment_language != current_language):

        current_speaker = fragment_speaker
        current_language = fragment_language

        print('---------------------')
        print(f'{current_speaker} ({current_language})')
        print('---------------------')

    #printing text
    print(fragment['words'][0]['text'])

---------------------
Daniel Warfield (en-us)
---------------------
Hello, my name is Daniel Warfield and I'm talking to the Bot.
Maybe.
Oh, thank God. Okay.
Cool. The money in the email.
Beautiful.
Okay.
Yeah.
All right.


# Checking if Meeting is Done

In [ ]:
def get_bot_info(api_key, bot_id, region='us-west-2'):

    url = f"https://us-west-2.recall.ai/api/v1/bot/{bot_id}/"

    headers = {
        "accept": "application/json",
        "Authorization": api_key
    }

    return requests.get(url, headers=headers)

response = get_bot_info(RECALL_API_KEY, bot_id)

# Should be 'done' if meeting is done
print(json.loads(response.text)['status_changes'][-1]['code'])

In [ ]:
# Should be 'done' if meeting is done
json.loads(response.text)['status_changes'][-1]['code']

'done'